# Machine Learning Essentials SS25 - Exercise Sheet 5

## Instructions
- `TODO`'s indicate where you need to complete the implementations.
- You may use external resources, but <b>write your own solutions</b>.
- Provide concise, but comprehensible comments to explain what your code does.
- Code that's unnecessarily extensive and/or not well commented will not be scored.

## Exercise 1

## Exercise 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# TODO: Import the stuff you need from torch and torchvision

""""
If you stay in ML-related fields, you will likely be working on a server or cluster. If you do so,
always remember to set the number of CPU (or even GPU) threads you're using, as Jupyter notebooks or Python scripts 
might sometimes use all available threads by default, which will lead to unhappy colleagues or classmates that
also want to use some of the threads.
"""
# Example of limiting CPU threads:
# import os
# os.environ["OMP_NUM_THREADS"] = "15"
# os.environ["MKL_NUM_THREADS"] = "15"
# torch.set_num_threads(15) # If you only want to use PyTorch threads

### 2.1

In [ ]:
# TODO: Define transformations
# Given statistics of training set:
mu_train = 0.286
std_train = 0.353
transform = 

# TODO: Load FashionMNIST train/testsets
train_dataset_full = 
test_dataset =

print(f"Full training dataset size: {len(train_dataset_full)}")
print(f"Test dataset size: {len(test_dataset)}")

In [ ]:
# TODO: Create 5x2 subplot grid w/ example image for each class

In [ ]:
# TODO: Create a validation set from the training set

### 2.2

In [ ]:
# TODO: Define your model architecture: A class called MLP that inherits from nn.Module
class MLP(nn.Module):
    ...

# TODO: Define appropriate loss
criterion = 

### 2.3

In [ ]:
BATCH_SIZE_DEFAULT =  # TODO: Set your default batch size. The capitalization is a convention used for global constants in Python

#TODO: Define DataLoaders for training, validation, and test sets

In [ ]:
def calculate_accuracy(outputs, labels):
    """
    Calculate accuracy given model outputs and true labels.
    """
    _, predicted = torch.max(outputs.data, 1) # Prediction = class with highest output probability
    total = labels.size(0) 
    correct = (predicted == labels).sum().item() #.item() converts a single-element tensor to a Python number ("scalar")
    return correct / total

def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    # Device configuration: if available use GPU (needs CUDA installed and GPU with PyTorch support), otherwise CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    print(f"Training on device: {device}")

    # TODO: Define training loop that for each epoch iterates over all mini-batches in the training set 
    # Record and return the training&validation loss and accuracy for each epoch

    return train_losses, val_losses, train_accuracies, val_accuracies

### 2.4

In [ ]:
# TODO: Define hyperparameter grid for tuning

# TODO: For each hyperparameter setting, instantiate model&optimizer, 
# train the model, and store the results for evaluation later

In [ ]:
# TODO: Plot evolution of validation accuracy for each hyperparameter setting

**TODO**: Justify which batch size and learning rate combination you will go with.

### 2.4

In [ ]:
# TODO: Train model w/ best hyperparameters for SGD and compare to default Adam optimizer

In [ ]:
# TODO: Plot "learning curves" of the best SGD model and the Adam model

**TODO**: Based on plots, compare (mini-batch) SGD and Adam, select overall best Model

In [ ]:
#TODO: Evaluate the best model on the test set, print the test/train/validation accuracy

**TODO**: Briefly discuss your results.
